## Install

In [ ]:
! pip install rdkit-pypi
! pip install deepchem
! pip install dgl 
! pip install ogb

In [ ]:
## to resolve the torch import error
# ! pip install -U numpy

## Import

In [ ]:
import numpy as np
import pandas
import time
import networkx as nx
import itertools
import scipy.sparse as sp
import random 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from rdkit.Chem import MACCSkeys
from rdkit import Chem

In [ ]:
import dgl
from dgl.nn import SAGEConv,GraphConv
import dgl.function as fn

In [ ]:
from sklearn.metrics import roc_auc_score
from ogb.linkproppred import Evaluator

In [ ]:
import copy

In [ ]:
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict)


## Read file

In [ ]:
!wget https://raw.githubusercontent.com/r-b-1-5/Public-files/main/drugIDandSMILES.csv


In [ ]:
csvFile = pandas.read_csv('./drugIDandSMILES.csv')
 
print(len(csvFile))
print(csvFile)

In [ ]:
drug_id = csvFile['Drug ID']
smiles = csvFile['SMILES']

## Globals


In [ ]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

## Form graph from molecule

In [ ]:
def smiles2graph(smiles_string):

    mol = Chem.MolFromSmiles(smiles_string)

    try:
        A = Chem.GetAdjacencyMatrix(mol)
        A = np.asmatrix(A)
        nnodes=len(A)
        nz = np.nonzero(A)
    except:
        return dgl.graph()
    # forming the graph using the adjacency matrix
    u1, v1 = list(nz[0]), list(nz[1])
    # print(sorted(u1)==sorted(v1))
    # print(u1)
    # print(v1)
    g = dgl.graph((u1, v1))
    bg = dgl.to_bidirected(g)

    # # atoms
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)

    # # bonds
    # num_bond_features = 3  # bond type, bond stereo, is_conjugated
    # if len(mol.GetBonds()) > 0: # mol has bonds
    #     edges_list = []
    #     edge_features_list = []
    #     for bond in mol.GetBonds():
    #         i = bond.GetBeginAtomIdx()
    #         j = bond.GetEndAtomIdx()

    #         edge_feature = bond_to_feature_vector(bond)

    #         # add edges in both directions
    #         edges_list.append((i, j))
    #         edge_features_list.append(edge_feature)
    #         edges_list.append((j, i))
    #         edge_features_list.append(edge_feature)

    #     # data.edge_index: Graph connectivity in COO format with shape [2, num_edges]
    #     edge_index = np.array(edges_list, dtype = np.int64).T

    #     # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
    #     edge_attr = np.array(edge_features_list, dtype = np.int64)

    # else:   # mol has no bonds
    #     edge_index = np.empty((2, 0), dtype = np.int64)
    #     edge_attr = np.empty((0, num_bond_features), dtype = np.int64)

    # print(edge_attr.shape, edge_index.shape, x.shape)
    bg.ndata['node_feat'] = torch.FloatTensor(x)
    # bg.edata['edge_feat'] = torch.tensor(edge_attr)

    # return graph 
    return bg

In [ ]:
mol1 = smiles2graph(smiles[0])
print(mol1)
mol2 = smiles2graph(smiles[1])
print(mol2)
# print(mol1.ndata['node_feat'])
# print(mol2.ndata['node_feat'])

In [ ]:
help(atom_to_feature_vector)

In [ ]:
# https://github.com/snap-stanford/ogb/blob/master/ogb/utils/features.py
maxnodes = 0
exception_count = 0
for _ in range(len(smiles)):
  try:
      mol_ = smiles2graph(smiles[_])
      maxnodes = max(maxnodes, mol_.num_nodes())
  except:
      exception_count += 1
      # print(smiles[_])


In [ ]:
print(maxnodes)
print(exception_count)

In [ ]:
# https://github.com/snap-stanford/ogb/blob/master/ogb/utils/features.py#L3

## VAE for Graph

In [ ]:
print(maxnodes)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Encoder, self).__init__()
        self.layersList = nn.ModuleList()
        self.layersList.append(SAGEConv(in_feats, out_feats, 'gcn'))
        self.layersList.append(SAGEConv(out_feats, out_feats, 'gcn'))
        self.attention = Attention(maxnodes,maxnodes)
        self.fc = nn.Linear( maxnodes*out_feats , 100)
        self.fc_mu = nn.Linear( 100 , 30)
        self.fc_var = nn.Linear( 100 , 30)
        self.relu = nn.ReLU()

    def forward(self, g, feats):
        temp = feats
        for L in self.layersList:
            temp = L(g,temp)
            temp = self.relu(temp)
        temp2 = temp
        dim1 = temp.shape[0]
        if maxnodes != dim1:
            padder = torch.zeros(maxnodes-dim1,3)
            temp = torch.cat([temp, padder], dim = 0)
        temp = self.attention(temp.t()).t()
        temp = torch.flatten(temp)
        temp = self.relu(self.fc(temp))
        mu = self.fc_mu(temp)
        # var = torch.exp(self.fc_var(temp))
        log_var = self.fc_var(temp)
        # return temp2, temp, mu, log_var
        return temp2, mu, log_var

class Attention(nn.Module):
    def __init__(self, in_feat,out_feat):
        super().__init__()             
        self.Q = nn.Linear(in_feat,out_feat) # Query
        self.K = nn.Linear(in_feat,out_feat) # Key
        self.V = nn.Linear(in_feat,out_feat) # Value
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        Q = self.Q(x)
        K = self.K(x)
        V = self.V(x)
        d = K.shape[0] # dimension of key vector
        QK_d = (Q @ K.T)/(d)**0.5
        prob = self.softmax(QK_d)
        attention = prob @ V
        return attention

class Decoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Decoder, self).__init__()
        self.layersList = nn.ModuleList()
        self.layersList.append(SAGEConv(in_feats, out_feats, 'gcn'))
        self.layersList.append(SAGEConv(out_feats, out_feats, 'gcn'))
        self.fc1 = nn.Linear( 30, 100)
        self.fc2 = nn.Linear( 100, maxnodes*in_feats)
        # self.attention = Attention(maxnodes, maxnodes)
        self.relu = nn.ReLU()
        

    def forward(self, g, res, hidden):
        temp = self.fc1(hidden)
        temp = self.fc2(temp)
        temp = torch.reshape(temp, (maxnodes, 3))
        # temp = self.attention(temp.t()).t()
        temp = temp[:res.shape[0]]
        temp += res
        for L in self.layersList:
            temp = L(g,temp)
            temp = self.relu(temp)
        return temp

class VariationalAutoEncoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(VariationalAutoEncoder, self).__init__()
        self.encoder = Encoder(in_feats, out_feats)
        self.decoder = Decoder(out_feats, in_feats)
        self.batchNormLayer = nn.BatchNorm1d(out_feats)

    def reparameterize(self, mu, log_var):
        # sampling
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, g, inputs):
        # temp, encoded, mu, log_var = self.encoder(g, inputs)
        temp, mu, log_var = self.encoder(g, inputs)
        hidden = self.reparameterize(mu, log_var)
        temp = self.batchNormLayer(temp)
        temp = self.decoder(g, temp, hidden)
        return temp

In [ ]:
set_seed(0)
g = smiles2graph(smiles[0])
g = dgl.add_self_loop(g)

model = VariationalAutoEncoder(g.ndata['node_feat'].shape[1], 3)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# loss_fn = nn.KLDivLoss() + nn.MSELoss()
loss_fn1 = nn.KLDivLoss() 
loss_fn2 = nn.MSELoss()

all_logits = []
for e in range(1000):
    pred = model(g, g.ndata['node_feat'])
    pred = F.log_softmax(pred, 1)
    loss = loss_fn1(pred, g.ndata['node_feat']) + loss_fn2(pred, g.ndata['node_feat'])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 50 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

## VAE Padding using Mask(Working)

In [ ]:
print(maxnodes)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Encoder, self).__init__()
        self.layersList = nn.ModuleList()
        self.layersList.append(SAGEConv(in_feats, out_feats, 'gcn'))
        self.layersList.append(SAGEConv(out_feats, out_feats, 'gcn'))
        self.attention = Attention(maxnodes,maxnodes)
        self.fc = nn.Linear(maxnodes*out_feats , 100)
        self.fc_mu = nn.Linear( 100 , 30)
        self.fc_var = nn.Linear( 100 , 30)
        self.relu = nn.ReLU()

    def forward(self, g, feats):
        temp = feats
        # print(temp.shape)
        for L in self.layersList:
            temp = L(g,temp)
            temp = self.relu(temp)
        temp2 = temp
        # print("e1")
        dim1 = temp.shape[0]
        if maxnodes != dim1:
            padder = torch.empty(maxnodes-dim1,3).fill_(-0.03)
            temp = torch.cat([temp, padder], dim = 0)
        mask = self.make_input_mask(temp)
        # print(mask)
        # print(temp.shape, mask.shape) # --> 551 * 9
        temp = self.attention(temp.t(), mask.t()).t()
        temp = torch.flatten(temp)
        # print("e2")
        temp = self.relu(self.fc(temp))
        mu = self.fc_mu(temp)
        # var = torch.exp(self.fc_var(temp))
        log_var = self.fc_var(temp)
        # return temp2, temp, mu, log_var
        return temp2, mu, log_var

    def make_input_mask(self, src):
        # using -100 for the padded indices
        src_mask = (src != -0.03)
        # print(src_mask.shape) # 551 * 9
        return src_mask

class Attention(nn.Module):
    def __init__(self, in_feat,out_feat):
        super().__init__()             
        self.Q = nn.Linear(in_feat,out_feat) # Query
        self.K = nn.Linear(in_feat,out_feat) # Key
        self.V = nn.Linear(in_feat,out_feat) # Value
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, mask):
        Q = self.Q(x)
        K = self.K(x)
        V = self.V(x)
        d = K.shape[0] # dimension of key vector
        QK_d = (Q @ K.T)/((d)**0.5)
        # print(QK_d.shape) # --> 551 * 551 
        prob = self.softmax(QK_d)
        attention = prob @ V
        # print(attention.shape) --> 551 * 9
        # print("sa1")
        ######
        if mask is not None:
            attention = attention.masked_fill(mask == 0, 0)
        ######
        # all zeros in the padded layers
        # print(attention[])
        return attention

class Decoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Decoder, self).__init__()
        self.layersList = nn.ModuleList()
        self.layersList.append(SAGEConv(in_feats, out_feats, 'gcn'))
        self.layersList.append(SAGEConv(out_feats, out_feats, 'gcn'))
        self.fc1 = nn.Linear( 30, 100)
        self.fc2 = nn.Linear( 100, maxnodes*in_feats)
        self.relu = nn.ReLU()
        

    def forward(self, g, res, hidden):
        temp = self.fc1(hidden)
        temp = self.fc2(temp)
        temp = torch.reshape(temp, (maxnodes, 3))
        temp = temp[:res.shape[0]]
        temp += res
        for L in self.layersList:
            temp = L(g,temp)
            temp = self.relu(temp)
        return temp

class VariationalAutoEncoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(VariationalAutoEncoder, self).__init__()
        self.encoder = Encoder(in_feats, out_feats)
        self.decoder = Decoder(out_feats, in_feats)
        self.batchNormLayer = nn.BatchNorm1d(out_feats)

    def reparameterize(self, mu, log_var):
        # sampling
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, g, inputs):
        # temp, encoded, mu, log_var = self.encoder(g, inputs)
        # print("1")
        # print(inputs.shape, input_mask.shape) --> 551 *9 
        temp, mu, log_var = self.encoder(g, inputs)
        # print("2")
        hidden = self.reparameterize(mu, log_var)
        # print("3")
        temp = self.batchNormLayer(temp)
        # print("4")
        temp = self.decoder(g, temp, hidden)
        # print("5")
        return temp

In [ ]:
set_seed(0)
g = smiles2graph(smiles[0])
g = dgl.add_self_loop(g)

model = VariationalAutoEncoder(g.ndata['node_feat'].shape[1], 3)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

loss_fn1 = nn.KLDivLoss() 
loss_fn2 = nn.MSELoss()

all_logits = []
for e in range(1000):
    pred = model(g, g.ndata['node_feat'])
    pred = F.log_softmax(pred, 1)
    loss = loss_fn1(pred, g.ndata['node_feat']) + loss_fn2(pred, g.ndata['node_feat'])
    # loss = loss_fn1(pred, g.ndata['node_feat'])
    # loss = loss_fn2(pred, g.ndata['node_feat'])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 50 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

    # break

## VAE for Graph(BATCH) (INCOMPLETE)

In [ ]:
print(maxnodes)

In [ ]:
print(drug_id)
print(smiles)

In [ ]:
from rdkit.Chem import SaltRemover

def smiles2graphNew(smiles_string):

    mol = Chem.MolFromSmiles(smiles_string)
    mol = SaltRemover.StripMol(mol)

    try:
        A = Chem.GetAdjacencyMatrix(mol)
        A = np.asmatrix(A)
        nnodes=len(A)
        nz = np.nonzero(A)
    except:
        return dgl.graph()
    # forming the graph using the adjacency matrix
    u1, v1 = list(nz[0]), list(nz[1])
    # print(sorted(u1)==sorted(v1))
    # print(u1)
    # print(v1)
    g = dgl.graph((u1, v1))
    bg = dgl.to_bidirected(g)

    # # atoms
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)

    
    bg.ndata['node_feat'] = torch.FloatTensor(x)
    bg = dgl.add_self_loop(bg)

    # return graph 
    return bg

In [ ]:
help(SaltRemover)

In [ ]:
# g = smiles2graph(smiles[0])
# g = dgl.add_self_loop(g)

from dgl.data import DGLDataset
class MoleculesDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='molecules')

    def process(self):
        # drug_id, smiles --> lists that store the values of the two

        self.graphs = []

        for i in range(len(drug_id)):
            g = smiles2graphNew(smiles[i])
            self.graphs.append(g)

        # n_graphs = len(self.graphs)
        # n_train = int(n_graphs * 0.6)
        # n_val = int(n_graphs * 0.2)
        # self.train_mask = torch.zeros(n_graphs, dtype=torch.bool)
        # val_mask = torch.zeros(n_graphs, dtype=torch.bool)
        # test_mask = torch.zeros(n_graphs, dtype=torch.bool)

        # train_mask[:n_train] = True
        # val_mask[n_train:n_train + n_val] = True
        # test_mask[n_train + n_val:] = True
        
        # self.graph.ndata['train_mask'] = train_mask
        # self.graph.ndata['val_mask'] = val_mask
        # self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graphs[i]

    def __len__(self):
        return len(self.graphs)

dataset = MoleculesDataset()
print(len(dataset))
graph = dataset[0]

print(graph)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Encoder, self).__init__()
        self.layersList = nn.ModuleList()
        self.layersList.append(SAGEConv(in_feats, out_feats, 'gcn'))
        self.layersList.append(SAGEConv(out_feats, out_feats, 'gcn'))
        self.attention = Attention(maxnodes,maxnodes)
        self.fc = nn.Linear( maxnodes*out_feats , 100)
        self.fc_mu = nn.Linear( 100 , 30)
        self.fc_var = nn.Linear( 100 , 30)
        self.relu = nn.ReLU()

    def forward(self, g, feats):
        temp = feats
        for L in self.layersList:
            temp = L(g,temp)
            temp = self.relu(temp)
        temp2 = temp
        dim1 = temp.shape[0]
        if maxnodes != dim1:
            padder = torch.zeros(maxnodes-dim1,3)
            temp = torch.cat([temp, padder], dim = 0)
        temp = self.attention(temp.t()).t()
        temp = torch.flatten(temp)
        temp = self.relu(self.fc(temp))
        mu = self.fc_mu(temp)
        # var = torch.exp(self.fc_var(temp))
        log_var = self.fc_var(temp)
        # return temp2, temp, mu, log_var
        return temp2, mu, log_var

class Attention(nn.Module):
    def __init__(self, in_feat,out_feat):
        super().__init__()             
        self.Q = nn.Linear(in_feat,out_feat) # Query
        self.K = nn.Linear(in_feat,out_feat) # Key
        self.V = nn.Linear(in_feat,out_feat) # Value
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        Q = self.Q(x)
        K = self.K(x)
        V = self.V(x)
        d = K.shape[0] # dimension of key vector
        QK_d = (Q @ K.T)/(d)**0.5
        prob = self.softmax(QK_d)
        attention = prob @ V
        return attention

class Decoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Decoder, self).__init__()
        self.layersList = nn.ModuleList()
        self.layersList.append(SAGEConv(in_feats, out_feats, 'gcn'))
        self.layersList.append(SAGEConv(out_feats, out_feats, 'gcn'))
        self.fc1 = nn.Linear( 30, 100)
        self.fc2 = nn.Linear( 100, maxnodes*in_feats)
        # self.attention = Attention(maxnodes, maxnodes)
        self.relu = nn.ReLU()
        

    def forward(self, g, res, hidden):
        temp = self.fc1(hidden)
        temp = self.fc2(temp)
        temp = torch.reshape(temp, (maxnodes, 3))
        # temp = self.attention(temp.t()).t()
        temp = temp[:res.shape[0]]
        temp += res
        for L in self.layersList:
            temp = L(g,temp)
            temp = self.relu(temp)
        return temp

class VariationalAutoEncoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(VariationalAutoEncoder, self).__init__()
        self.encoder = Encoder(in_feats, out_feats)
        self.decoder = Decoder(out_feats, in_feats)
        self.batchNormLayer = nn.BatchNorm1d(out_feats)

    def reparameterize(self, mu, log_var):
        # sampling
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, g, inputs):
        # temp, encoded, mu, log_var = self.encoder(g, inputs)
        temp, mu, log_var = self.encoder(g, inputs)
        hidden = self.reparameterize(mu, log_var)
        temp = self.batchNormLayer(temp)
        temp = self.decoder(g, temp, hidden)
        return temp 

In [ ]:
set_seed(0)
# g = smiles2graph(smiles[0])
# g = dgl.add_self_loop(g)
# 155*9 --> 155*3 -> 100
# x*9 --> x*3 --> maxnodes*3 --> 100 
model = VariationalAutoEncoder(g.ndata['node_feat'].shape[1], 3)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# loss_fn = nn.KLDivLoss() + nn.MSELoss()
loss_fn1 = nn.KLDivLoss() 
loss_fn2 = nn.MSELoss()

all_logits = []
for e in range(1000):
    pred = model(g, g.ndata['node_feat'])
    pred = F.log_softmax(pred, 1)
    loss = loss_fn1(pred, g.ndata['node_feat']) + loss_fn2(pred, g.ndata['node_feat'])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 50 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))